# 구조화 V1

In [ ]:
import pandas as pd
import re
import shutil
from pandasai import Agent, SmartDataframe
import pandas as pd
import panel as pn
import base64
from dotenv import load_dotenv
from time import sleep
load_dotenv()

# pd.set_option('display.max_rows', 20)
# pd.set_option('display.max_columns', 20)


def split_dataframe_by_sum(df, column_name):
    """
    주어진 DataFrame을 특정 열에서 '합계'를 기준으로 분할합니다.

    Parameters:
    df (pd.DataFrame): 분할할 DataFrame.
    column_name (str): '합계'를 검색할 열의 이름.

    Returns:
    list: '합계'를 포함한 각 부분으로 분할된 DataFrame의 리스트.
    """
    df.index = range(len(df))

    # 해당 열에서 '합계'가 있는 인덱스 찾기
    indices = df[df[column_name] == '합계'].index

    # DataFrame을 분할
    dataframes = []
    start_idx = 0

    for idx in indices:
        # '합계'를 포함하는 행까지 DataFrame 슬라이스
        dataframes.append(df.iloc[start_idx:idx+1])
        start_idx = idx + 1  # 다음 분할을 위해 시작 인덱스를 '합계' 다음 행으로 설정

    # 마지막 인덱스부터 DataFrame 끝까지 추가
    if start_idx < len(df):  # 마지막 '합계' 이후에도 데이터가 남아 있는 경우
        dataframes.append(df.iloc[start_idx:])

    return dataframes[0]

def split_dataframe_by_date(df, column_name):
    """
    주어진 DataFrame을 특정 열에서 '날짜'를 기준으로 분할합니다.

    Parameters:
    df (pd.DataFrame): 분할할 DataFrame.
    column_name (str): '날짜'를 검색할 열의 이름.

    Returns:
    list: '날짜'를 포함한 각 부분으로 분할된 DataFrame의 리스트.
    """
    df.index = range(len(df))
    df.columns = df.columns.str.replace('\n', '', regex=False)


    # 해당 열에서 '날짜'가 있는 인덱스 찾기
    indices = df[df[column_name] == '날짜'].index

    # DataFrame을 분할
    dataframes = []
    start_idx = 0

    for idx in indices+1:
        # '날짜'를 포함하는 행까지 DataFrame 슬라이스
        dataframes.append(df.iloc[start_idx:idx+1])
        start_idx = idx + 1  # 다음 분할을 위해 시작 인덱스를 '날짜' 다음 행으로 설정

    # 마지막 인덱스부터 DataFrame 끝까지 추가
    if start_idx < len(df):  # 마지막 '날짜' 이후에도 데이터가 남아 있는 경우
        dataframes.append(df.iloc[start_idx:])

    return dataframes[1]


# NaN 값을 이전 값으로 채우기
def fill_date(df):
    df['날짜'] = df['날짜'].fillna(method='ffill')

    return df


def convert_date_and_format(df, column, month):
    # 월을 나타내는 한글과 숫자를 매핑하는 딕셔너리
    month_dict = {'1월': '01', '2월': '02', '3월': '03', '4월': '04', '5월': '05', '6월': '06', 
                  '7월': '07', '8월': '08', '9월': '09', '10월': '10', '11월': '11', '12월': '12'}
    
    # 데이터 프레임의 복사본을 만듭니다.
    df_copy = df.copy()
    
    # 가정: df는 데이터 프레임, column은 변경하려는 열
    def convert_date(date, month):
        date = date.strip()
        if '야' in date:
            return '2023' + month_dict[month] + date.replace('야', '').replace('()', '').zfill(2) + ' 22:00'
        else:
            return '2023' + month_dict[month] + date.replace('()', '').zfill(2) + ' 12:00'

    df_copy[column] = df_copy[column].astype(str).apply(lambda x: convert_date(x, month))

    # 날짜 열의 데이터 타입을 datetime으로 변경
    df_copy[column] = pd.to_datetime(df_copy[column], format='%Y%m%d %H:%M', errors='coerce')
    return df_copy

def decompose(df, month):
    # '1호기' 열에서 '2호기' 정확히 매치되는 패턴 검색
    df = df.rename(columns = {'1호기': '날짜'})
    pattern = r'^2호기$'
    mask = df['날짜'].apply(lambda x: bool(re.search(pattern, str(x))))
    indices = df[mask].index
    
    # DataFrame을 분할
    dataframes = []
    start_idx = 0
    
    for idx in indices:
        # 현재 인덱스를 새 DataFrame의 시작으로 포함
        dataframes.append(df.iloc[start_idx:idx])
        start_idx = idx
    
    # 마지막 인덱스부터 DataFrame 끝까지 추가
    dataframes.append(df.iloc[start_idx:])

    FirstUnit = dataframes[0]
    SecondUnit = dataframes[1]


    # 1호기 소계, 2호기 소계 추출
    monthly_first = SecondUnit[SecondUnit['날짜']=='1호기소계']
    montyly_second = SecondUnit[SecondUnit['날짜']=='2호기소계']

    # Dataframe 정규화
    FirstUnit = split_dataframe_by_sum(FirstUnit, '날짜')
    FirstUnit = split_dataframe_by_date(FirstUnit, '날짜')
    
    SecondUnit = split_dataframe_by_sum(SecondUnit, '날짜')
    SecondUnit = split_dataframe_by_date(SecondUnit, '날짜')

    # NaN 값을 이전 값으로 채워주기
    FirstUnit = fill_date(FirstUnit)
    SecondUnit = fill_date(SecondUnit)

    # 합계 Dataframe 추출
    FirstUnitTotalSum = FirstUnit[FirstUnit['생산규격']=='합계']
    SecondUnitTotalSum = SecondUnit[SecondUnit['생산규격']=='합계']

    # 날짜 생성 및 포맷 변경
    FirstUnitTotalSum = convert_date_and_format(FirstUnitTotalSum, '날짜', month)
    SecondUnitTotalSum = convert_date_and_format(SecondUnitTotalSum, '날짜', month)
    
    return [FirstUnit, SecondUnit, monthly_first, montyly_second, FirstUnitTotalSum, SecondUnitTotalSum]




def transform_column_name(df):
    df_columns = {
    "날짜": "Date",
    "생산규격": "Production_Standards",
    "현재원": "Current_Staff",
    "작업시간(HR)": "Work_Hours",
    "총투입시간(HR)": "Total_Input_Hours",
    "총작업시간(분)": "Total_Work_Time",
    "휴지시간(비가동HR)": "Idle_Time",
    "순작업시간(가동HR)": "Net_Work_Time",
    "가동율(%)": "Operational_Rate",
    "양품율(%)": "Quality_Rate",
    "생산량(ton)": "Production",
    "투입량(ton)": "Input_Amount",
    "출고량(ton)": "Shipment_Amount",
    "생산총길이(M)": "Total_Production_Length",
    "UPH(M/HR)": "Units_Per_Hour",
    "SPH(천원/HR)": "Sales_Per_Hour",
    "TPH(TON/HR)": "Tons_Per_Hour",
    "MPH(M/HR)": "Meters_Per_Hour"}

    df.rename(columns=df_columns, inplace=True)
    return df

def delete_useless_columns(df):
    df = df.dropna(axis=1, how='all')
    df = df.dropna()
    return df

def index_reset(df):
    df.index = range(len(df))
    df = transform_column_name(df)
    df = delete_useless_columns(df)
    df = df.convert_dtypes()
    return df


def index_reset_v2(df):
    df.index = range(len(df))
    df = transform_column_name(df)
    df = df.convert_dtypes()
    return df

def monthly_df_maker(excel_sheets : dict) -> dict:
    '''
    montly_dict = {'1월' : montly_List, '2월' : montly_List, ...}
    montly_List = [1호기_df, 2호기_df, 1호기 소계, 2호기 소계, 1호기 합계, 2호기 합계]
    '''
    monthly_dict = {}

    try:
        for sheet in excel_sheets:
            result = decompose(excel_sheets[sheet],sheet)
            monthly_dict[sheet] = result
    except Exception as e :
        pass
        # print(sheet)
        # print(e)

    first_unit_df = pd.DataFrame()
    second_unit_df = pd.DataFrame()
    monthly_first = pd.DataFrame()
    monthly_second = pd.DataFrame()
    first_unit_total_sum = pd.DataFrame()
    second_unit_total_sum = pd.DataFrame()
    
    for key in monthly_dict:
        first_unit_df = pd.concat([first_unit_df, monthly_dict[key][0]])
        second_unit_df = pd.concat([second_unit_df, monthly_dict[key][1]])
        monthly_first = pd.concat([monthly_first, monthly_dict[key][2]])
        monthly_second = pd.concat([monthly_second, monthly_dict[key][3]])
        first_unit_total_sum = pd.concat([first_unit_total_sum, monthly_dict[key][4]])
        second_unit_total_sum = pd.concat([second_unit_total_sum, monthly_dict[key][5]]) 

    # Index Reset
    first_unit_df = index_reset_v2(first_unit_df)
    second_unit_df = index_reset_v2(second_unit_df)   
    monthly_first = index_reset(monthly_first)
    monthly_second = index_reset(monthly_second)
    first_unit_total_sum = index_reset(first_unit_total_sum)
    second_unit_total_sum = index_reset(second_unit_total_sum)
    first_unit_total_sum.drop('Production_Standards', axis=1, inplace=True)
    second_unit_total_sum.drop('Production_Standards', axis=1, inplace=True)

    return [first_unit_df, second_unit_df, monthly_first, monthly_second, first_unit_total_sum, second_unit_total_sum]


# 사용할지 모르는 함수
def save_to_excel(monthly_list):
    for idx, df in enumerate(monthly_list):
        if idx == 0:
            df.to_excel('1호기.xlsx', index=False)
        elif idx == 1:
            df.to_excel('2호기.xlsx', index=False)
        elif idx == 2:
            df.to_excel('1호기 소계.xlsx', index=False)
        elif idx == 3:
            df.to_excel('2호기 소계.xlsx', index=False)
        elif idx == 4:
            df.to_excel('1호기 합계.xlsx', index=False)
        elif idx == 5:
            df.to_excel('2호기 합계.xlsx', index=False)
        
    return None
            
# 월별 인적자원 개요
def hr_monthly(sheets):
        graph_color = 'light blue'
        line_color = 'blue'
        monthly_list = monthly_df_maker(sheets)

        agent_unit_1 = SmartDataframe(monthly_list[4],
                config={
                        "open_charts": False,
                        'save_charts': True,
                        'max_retries' : 10,
                        'verbose' : True},
                description="You are a data analysis agent. Your main goal is to help non-technical users to analyze data")
        sleep(1)
        agent_unit_2 = SmartDataframe(monthly_list[4],
                config={
                        "open_charts": False,
                        'save_charts': True,
                        'max_retries' : 10,
                        'verbose' : True},
                description="You are a data analysis agent. Your main goal is to help non-technical users to analyze data")
        sleep(1)
        


        ## 월별 순 작업시간 합계 unit 1
        NetWorkHoursPerMonth_unit_1 = agent_unit_1.chat(f"""Add up the total Net_Work_Time for each month and plot it in a bar plot\n
                        Set Title "Net Work Hours per Month(UNIT_1)"\n
                        The x-axis is filled in with all of each month\n
                        Drawing a dot and line graph on top of a bar graph\n
                        delete legend\n
                        Draw the {graph_color} color
                """)
        
        sleep(1)
        ## 월별 순 작업시간 합계 unit 2
        NetWorkHoursPerMonth_unit_2 = agent_unit_2.chat(f"""Add up the total Net_Work_Time for each month and plot it in a bar plot\n
                        Set Title "Net Work Hours per Month(UNIT_2)"\n
                        The x-axis is filled in with all of each month\n
                        Drawing a dot and line graph on top of a bar graph\n
                        delete legend\n
                        Draw the {graph_color} color
                """)
        sleep(1)
        ## 월별 가동율 평균 Unit 1
        AverageOperationalRatePerMonth_unit_1 = agent_unit_1.chat(f"""
                        Plot the average of Operational_Rate by month as a line graph\n
                        Set Title "Aaverge Operational_Rate per Month(UNIT_1)"\n
                        Draw the {graph_color} color.\n
                        save the chart as "AverageOperationalRatePerMonth"
                """)
        sleep(1)
        ## 월별 가동율 평균 unit 2

        AverageOperationalRatePerMonth_unit_2 = agent_unit_1.chat(f"""
                        Plot the average of Operational_Rate by month as a line graph\n
                        Set Title "Aaverge Operational_Rate per Month(UNIT_2)"\n
                        Draw the {graph_color} color.\n
                        save the chart as "AverageOperationalRatePerMonth"
                """)
        sleep(1)

        ## 월별 총 투입시간 합계 unit 1
        Total_unit_1 = agent_unit_1.chat(f"""
                        Add up the total Total_Input_Hours for each month and plot it in a bar plot\n
                        Set Title "Total Input Hours per Month(UNIT_1)"\n
                        The x-axis is filled in with all of each month\n
                        Drawing a dot and line graph on top of a bar graph\n
                        Draw the {graph_color} color
                        save the chart as "TotalInputHoursPerMonth"
                """)
        sleep(1)
        ## 월별 총 투입시간 합계 unit 2
        Total_unit_2 = agent_unit_2.chat(f"""
                        Add up the total Total_Input_Hours for each month and plot it in a bar plot\n
                        Set Title "Total Input Hours per Month(UNIT_2)"\n
                        The x-axis is filled in with all of each month\n
                        Drawing a dot and line graph on top of a bar graph\n
                        Draw the {graph_color} color
                        save the chart as "TotalInputHoursPerMonth"
                """)
        sleep(1)
        png_files = [NetWorkHoursPerMonth_unit_1, NetWorkHoursPerMonth_unit_2, 
                     AverageOperationalRatePerMonth_unit_1, AverageOperationalRatePerMonth_unit_2,
                     Total_unit_1, Total_unit_2]
        
        return png_files

# 각 월 인적자원 개요
def hr_each_month(sheets):
        graph_color = 'light blue'
        line_color = 'blue'
        month_list = ["january", "february", "march", "april", "may", "june", "july", "august", "september", "october", "november", "december"]
        hr_monthly_dict = {}
        monthly_list = monthly_df_maker(sheets)
        for month in month_list:
            

            agent_unit_1 = SmartDataframe(monthly_list[4],
                    config={
                            "open_charts": False,
                            'save_charts': True,
                            'max_retries' : 10,
                            'verbose' : True},
                    description="You are a data analysis agent. Your main goal is to help non-technical users to analyze data")

            agent_unit_2 = SmartDataframe(monthly_list[4],
                    config={
                            "open_charts": False,
                            'save_charts': True,
                            'max_retries' : 10,
                            'verbose' : True},
                    description="You are a data analysis agent. Your main goal is to help non-technical users to analyze data")
            
            month_list = []
            
                    ## 월별 순 작업시간 합계 unit 1
            NetWorkHoursPerMonth_unit_1 = agent_unit_1.chat(f"""
                                    In a dataframe, plot a time series graph with x as Date and y as Net_Work_Time for the month of {month}.\n
                                    Set Title "Net Work Hours(UNIT_1)"\n
                                    Drawing a dot and line graph on top of a bar graph\n
                                    delete legend\n
                                    Draw the {graph_color} color
                                    save the chart as "NetWorkHoursPerMonth_unit_1"
                    """)
            sleep(1)
            # ## 월별 순 작업시간 합계 unit 2
            NetWorkHoursPerMonth_unit_2 = agent_unit_2.chat(f"""
                                    In a dataframe, plot a time series graph with x as Date and y as Net_Work_Time for the month of {month}.\n
                                    Set Title "Net Work Hours(UNIT_2)"\n
                                    Drawing a dot and line graph on top of a bar graph\n
                                    delete legend\n
                                    Draw the {graph_color} color
                                    save the chart as "NetWorkHoursPerMonth_unit_2"
                    """)
            sleep(1)
            # ## 월별 가동율 평균 Unit 1
            AverageOperationalRatePerMonth_unit_1 = agent_unit_1.chat(f"""
                                    In a dataframe, plot a time series graph with x as Date and y as Operational_Rate for the month of {month}.\n
                                    Set Title "Operational_Rate(UNIT_1)"\n
                                    Draw the {graph_color} color.\n
                                    save the chart as "AverageOperationalRatePerMonth"
                    """)
            sleep(1)
            # ## 월별 가동율 평균 unit 2
            AverageOperationalRatePerMonth_unit_2 = agent_unit_2.chat(f"""
                                    In a dataframe, plot a time series graph with x as Date and y as Operational_Rate for the month of {month}.\n
                                    Set Title "Operational_Rate(UNIT_2)"\n
                                    Draw the {graph_color} color.\n
                                    save the chart as "AverageOperationalRatePerMonth"
                    """)
            
            month_list.append(NetWorkHoursPerMonth_unit_1)
            month_list.append(NetWorkHoursPerMonth_unit_2)
            month_list.append(AverageOperationalRatePerMonth_unit_1)
            month_list.append(AverageOperationalRatePerMonth_unit_2)
                    
            hr_monthly_dict[month] = month_list
        
        return hr_monthly_dict

# 월별 생산관련 개요
def pr_monthly(sheets):
    monthly_list = monthly_df_maker(sheets)
    graph_color = 'light blue'
    line_color = 'blue'
    agent_unit_1 = SmartDataframe(monthly_list[4],
                config={
                        "open_charts": False,
                        'save_charts': True,
                        'max_retries' : 10,
                        'verbose' : True},
                description="You are a data analysis agent. Your main goal is to help non-technical users to analyze data")

    agent_unit_2 = SmartDataframe(monthly_list[4],
                config={
                        "open_charts": False,
                        'save_charts': True,
                        'max_retries' : 10,
                        'verbose' : True},
                description="You are a data analysis agent. Your main goal is to help non-technical users to analyze data")

    ProductionPerMonth_unit_1 = agent_unit_1.chat(f"""
                        Add up the total Production for each month and plot it in a bar plot\n
                        Set Title "Production per Month(UNIT_1)"\n
                        The x-axis is filled in with all of each month\n
                        Drawing a dot and line graph on top of a bar graph\n
                        Draw the {graph_color} color
                        save the chart as "ProductionPerMonth"
                    """)
    sleep(1)
    ProductionPerMonth_unit_2 = agent_unit_2.chat(f"""
                        Add up the total Production for each month and plot it in a bar plot\n
                        Set Title "Production per Month(UNIT_2)"\n
                        The x-axis is filled in with all of each month\n
                        Drawing a dot and line graph on top of a bar graph\n
                        Draw the {graph_color} color
                        save the chart as "ProductionPerMonth"
                    """)
    sleep(1)
    ## 월별 (생산량 합계)/순작업시간

    WorkEfficiency_unit_1 = agent_unit_1.chat(f"""
                        Plot a line graph of the sum of production/(Current_Staff*Net_Work_Time) per month.\n
                        Set Title "production/(Current_Staff*Net_Work_Time)_(UNIT_1)"\n
                        The x-axis is filled in with all of each month\n
                        Draw the {line_color} color
                        save the chart as "WorkEfficiency"
                    """)
    sleep(1)

    WorkEfficiency_unit_2 = agent_unit_2.chat(f"""
                        Plot a line graph of the sum of production/(Current_Staff*Net_Work_Time) per month.\n
                        Set Title "production/(Current_Staff*Net_Work_Time)_(UNIT_2)"\n
                        The x-axis is filled in with all of each month\n
                        Draw the {line_color} color
                        save the chart as "WorkEfficiency"
                    """)
    sleep(1)
    ## 월별 (생산량 합계)/총 투입시간
    ProductionCostEfficiency_unit_1= agent_unit_1.chat(f"""
                        Plot a line graph of the sum of production/(Total_Input_Hours * 60) per month.\n
                        Set Title "production/(Total_Input_Hours)_(UNIT_1)"\n
                        The x-axis is filled in with all of each month\n
                        Draw the {line_color} color
                        save the chart as "ProductionCostEfficiency"
                    """)
    sleep(1)
    ProductionCostEfficiency_unit_2= agent_unit_2.chat(f"""
                        Plot a line graph of the sum of production/(Total_Input_Hours * 60) per month.\n
                        Set Title "production/(Total_Input_Hours)_(UNIT_2)"\n
                        The x-axis is filled in with all of each month\n
                        Draw the {line_color} color
                        save the chart as "ProductionCostEfficiency"
                    """)
    sleep(1)
    ## 월별 Shipment_Amount 합계
    ShipmentPerMonth_unit_1 = agent_unit_1.chat(f"""    
                        Add up the total Shipment_Amount for each month and plot it in a bar plot\n
                        Set Title "Shipment_Amount per Month(UNIT_1)"\n
                        The x-axis is filled in with all of each month\n
                        Drawing a dot and line graph on top of a bar graph\n
                        Draw the {graph_color} color
                        save the chart as "ShipmentPerMonth"
                    """)

    sleep(1)
    ShipmentPerMonth_unit_2 = agent_unit_2.chat(f"""    
                        Add up the total Shipment_Amount for each month and plot it in a bar plot\n
                        Set Title "Shipment_Amount per Month(UNIT_2)"\n
                        The x-axis is filled in with all of each month\n
                        Drawing a dot and line graph on top of a bar graph\n
                        Draw the {graph_color} color
                        save the chart as "ShipmentPerMonth"
                    """)

    prduction_ov = [ProductionPerMonth_unit_1,ProductionPerMonth_unit_2, 
                    WorkEfficiency_unit_1,WorkEfficiency_unit_2,
                    ProductionCostEfficiency_unit_1,ProductionCostEfficiency_unit_2,
                    ShipmentPerMonth_unit_1,ShipmentPerMonth_unit_2]
    
    return prduction_ov

# 각 월 생산관련
def pr_each_month(sheets):
        graph_color = 'light blue'
        line_color = 'blue'
        month_list = ["january", "february", "march", "april", "may", "june", "july", "august", "september", "october", "november", "december"]
        pr_monthly_dict = {}
        for month in month_list:
                
                monthly_list = monthly_df_maker(sheets)

                agent_unit_1 = SmartDataframe(monthly_list[4],
                config={
                        "open_charts": False,
                        'save_charts': True,
                        'max_retries' : 10,
                        'verbose' : True},
                description="You are a data analysis agent. Your main goal is to help non-technical users to analyze data")

                agent_unit_2 = SmartDataframe(monthly_list[4],
                config={
                        "open_charts": False,
                        'save_charts': True,
                        'max_retries' : 10,
                        'verbose' : True},
                description="You are a data analysis agent. Your main goal is to help non-technical users to analyze data")
        
                month_list = []
        
                ProductionPerMonth_unit_1 = agent_unit_1.chat(f"""
                                In a dataframe, plot a time series graph with x as Date and y as Production for the month of {month}.\n
                                Set Title "Production(UNIT_1)"\n
                                Drawing a dot and line graph on top of a bar graph\n
                                Draw the {graph_color} color
                                save the chart as "ProductionPerMonth"
                                """)
                sleep(1)
                ProductionPerMonth_unit_2 = agent_unit_2.chat(f"""
                                In a dataframe, plot a time series graph with x as Date and y as Production for the month of {month}.\n
                                Set Title "Production(UNIT_2)"\n
                                The x-axis is filled in with all of each month\n
                                Drawing a dot and line graph on top of a bar graph\n
                                Draw the {graph_color} color
                                save the chart as "ProductionPerMonth"
                                """)
                sleep(1)
                ## 월별 (생산량 합계)/순작업시간

                WorkEfficiency_unit_1 = agent_unit_1.chat(f"""
                                In a dataframe, plot a time series graph with x as Date and y as production/(Current_Staff*Net_Work_Time) for the month of {month}.\n
                                Set Title "production/(Current_Staff*Net_Work_Time)_(UNIT_1)"\n
                                Draw the {line_color} color
                                save the chart as "WorkEfficiency"
                                """)
                sleep(1)

                WorkEfficiency_unit_2 = agent_unit_2.chat(f"""
                                In a dataframe, plot a time series graph with x as Date and y as production/(Current_Staff*Net_Work_Time) for the month of {month}.\n
                                Set Title "production/(Current_Staff*Net_Work_Time)_(UNIT_2)"\n
                                Draw the {line_color} color
                                save the chart as "WorkEfficiency"
                                """)
                sleep(1)
                ## 월별 (생산량 합계)/총 투입시간
                ProductionCostEfficiency_unit_1= agent_unit_1.chat(f"""
                                In a dataframe, plot a time series graph with x as Date and y as production/(Total_Input_Hours * 60) for the month of {month}.\n
                                Set Title "production/(Total_Input_Hours)_(UNIT_1)"\n
                                Draw the {line_color} color
                                save the chart as "ProductionCostEfficiency"
                                """)
                sleep(1)
                ProductionCostEfficiency_unit_2= agent_unit_2.chat(f"""
                                In a dataframe, plot a time series graph with x as Date and y as production/(Total_Input_Hours * 60) for the month of {month}.\n
                                Set Title "production/(Total_Input_Hours)_(UNIT_2)"\n
                                Draw the {line_color} color
                                save the chart as "ProductionCostEfficiency"
                                """)
                sleep(1)
                ## 월별 Shipment_Amount 합계
                ShipmentPerMonth_unit_1 = agent_unit_1.chat(f"""    
                                In a dataframe, plot a time series graph with x as Date and y as Shipment_Amount for the month of {month}.\n
                                Set Title "Shipment_Amount(UNIT_1)"\n
                                Drawing a dot and line graph on top of a bar graph\n
                                Draw the {graph_color} color
                                save the chart as "ShipmentPerMonth"
                                """)

                sleep(1)
                ShipmentPerMonth_unit_2 = agent_unit_2.chat(f"""    
                                In a dataframe, plot a time series graph with x as Date and y as Shipment_Amount for the month of {month}.\n
                                Set Title "Shipment_Amount per Month(UNIT_2)"\n
                                Drawing a dot and line graph on top of a bar graph\n
                                Draw the {graph_color} color
                                save the chart as "ShipmentPerMonth"
                                """)
                
                month_list.append(ProductionPerMonth_unit_1)
                month_list.append(ProductionPerMonth_unit_2)
                month_list.append(WorkEfficiency_unit_1)
                month_list.append(WorkEfficiency_unit_2)
                month_list.append(ProductionCostEfficiency_unit_1)
                month_list.append(ProductionCostEfficiency_unit_2)
                month_list.append(ShipmentPerMonth_unit_1)
                month_list.append(ShipmentPerMonth_unit_2)
                        
                pr_monthly_dict[month] = month_list
                
        return pr_monthly_dict
        
def make_dashboard(sheets):
    PRIMARY_COLOR = "#0072B5"
    SECONDARY_COLOR = "#B54300"
    pn.extension(design="material", sizing_mode="stretch_width")

    # 파일 경로
    monthly_list = monthly_df_maker(sheets)
    png_files = hr_monthly(sheets)
    prduction_ov = pr_monthly(sheets)
    hr_monthly_dict = hr_each_month(sheets)
    pr_monthly_dict = pr_each_month(sheets)
    # 인적자원 png 읽기 및 인코딩
    encoded_pngs = []
    for png_file in png_files:
        # PNG 파일 읽기
        with open(png_file, 'rb') as f:
            png_data = f.read()

        # PNG 데이터 인코딩
        encoded_png = base64.b64encode(png_data).decode('utf-8')
        encoded_pngs.append(encoded_png)

    # 3x2 배열로 이미지 배치
    hr_overview_images = '<table>\n'
    for i in range(0, len(encoded_pngs), 2):
        hr_overview_images += '<tr>\n'
        for j in range(2):
            if i + j < len(encoded_pngs):
                hr_overview_images += f'<td><img src="data:image/png;base64,{encoded_pngs[i+j]}" alt="PNG 이미지"></td>\n'
        hr_overview_images += '</tr>\n'
    hr_overview_images += '</table>'

    # 생산관련 PNG 파일 읽기 및 인코딩
    encoded_pngs_prd = []
    for png_file in prduction_ov:
        with open(png_file, 'rb') as f:
            png_data = f.read()
        encoded_png = base64.b64encode(png_data).decode('utf-8')
        encoded_pngs_prd.append(encoded_png)

    # 생산관련 개요에 이미지 추가
    prd_overview_images = '<table>\n'
    for i in range(0, len(encoded_pngs_prd), 2):
        prd_overview_images += '<tr>\n'
        for j in range(2):
            if i + j < len(encoded_pngs_prd):
                prd_overview_images += f'<td><img src="data:image/png;base64,{encoded_pngs_prd[i+j]}" alt="PNG 이미지"></td>\n'
        prd_overview_images += '</tr>\n'
    prd_overview_images += '</table>'

    # 각 월에 대한 이미지 추가
    hr_monthly_images = {}
    for month, png_files in hr_monthly_dict.items():
        encoded_pngs = []
        for png_file in png_files:
            with open(png_file, 'rb') as f:
                png_data = f.read()
            encoded_png = base64.b64encode(png_data).decode('utf-8')
            encoded_pngs.append(encoded_png)
        hr_monthly_images[month] = '\n'.join([f'<img src="data:image/png;base64,{encoded_png}" alt="PNG 이미지">' for encoded_png in encoded_pngs])

    

    # 각 월에 대한 이미지 추가
    pr_monthly_images = {}
    for month, png_files in pr_monthly_dict.items():
        encoded_pngs = []
        for png_file in png_files:
            with open(png_file, 'rb') as f:
                png_data = f.read()
            encoded_png = base64.b64encode(png_data).decode('utf-8')
            encoded_pngs.append(encoded_png)
        pr_monthly_images[month] = '\n'.join([f'<img src="data:image/png;base64,{encoded_png}" alt="PNG 이미지">' for encoded_png in encoded_pngs])

    # DataFrame을 HTML로 변환
    df1_html = monthly_list[2].to_html()  # 1호기 소계
    df2_html = monthly_list[3].to_html()  # 2호기 소계
    df_total_html_unit_1 = monthly_list[4].to_html()  # 1호기 합계
    df_total_html_unit_2 = monthly_list[5].to_html()  # 2호기 합계




    css_style = """
    .nav-item {
        flex-grow: 0;
        flex-shrink: 0;
    }
    td {
        white-space: nowrap;
    }
    """

    html_template = f'''
    <!DOCTYPE html>
    <html>
    <head>
        <title>TPC Global</title>
        <!-- Bootstrap CSS -->
        <link href="https://stackpath.bootstrapcdn.com/bootstrap/4.3.1/css/bootstrap.min.css" rel="stylesheet">
        <style>
            {css_style}
        </style>
    </head>
    <body>
        <h1>TPC Global</h1>
        <div class="container-fluid">
            <ul class="nav nav-tabs">
                <li class="nav-item">
                    <a class="nav-link active" data-toggle="tab" href="#hr">인적자원</a>
                </li>
                <li class="nav-item">
                    <a class="nav-link" data-toggle="tab" href="#production">생산관련</a>
                </li>
                <li class="nav-item">
                    <a class="nav-link" data-toggle="tab" href="#subtotal1">1호기 소계</a>
                </li>
                <li class="nav-item">
                    <a class="nav-link" data-toggle="tab" href="#subtotal2">2호기 소계</a>
                </li>
                <li class="nav-item">
                    <a class="nav-link" data-toggle="tab" href="#total1">1호기 일별 합계</a>
                </li>
                <li class="nav-item">
                    <a class="nav-link" data-toggle="tab" href="#total2">2호기 일별 합계</a>
                </li>
            </ul>
            <div class="tab-content">
                <div class="tab-pane active" id="hr">
                    <ul class="nav nav-tabs">
                        <li class="nav-item">
                            <a class="nav-link active" data-toggle="tab" href="#hr_overview">개요</a>
                        </li>
                        {"".join([f'<li class="nav-item"><a class="nav-link" data-toggle="tab" href="#hr_{month}">{month}</a></li>' for month in hr_monthly_dict.keys()])}
                    </ul>
                    <div class="tab-content">
                        <div class="tab-pane active" id="hr_overview">
                            {hr_overview_images}
                        </div>
                        {"".join([f'<div class="tab-pane" id="hr_{month}">{hr_monthly_images[month]}</div>' for month in hr_monthly_dict.keys()])}
                    </div>
                </div>
                <div class="tab-pane" id="production">
                    <ul class="nav nav-tabs">
                        <li class="nav-item">
                            <a class="nav-link active" data-toggle="tab" href="#production_overview">개요</a>
                        </li>
                        {"".join([f'<li class="nav-item"><a class="nav-link" data-toggle="tab" href="#production_{i}">{i}월</a></li>' for i in range(1, 13)])}
                    </ul>
                    <div class="tab-content">
                        <div class="tab-pane active" id="production_overview">
                            {prd_overview_images}
                        </div>
                        {"".join([f'<div class="tab-pane" id="production_{i}"><img src="data:image/png;base64,{encoded_pngs[i-1 if i-1 < len(encoded_pngs) else 0]}" alt="PNG 이미지{i}"></div>' for i in range(1, 13)])}
                    </div>
                </div>
                <div class="tab-pane" id="subtotal1">
                    {df1_html}
                </div>
                <div class="tab-pane" id="subtotal2">
                    {df2_html}
                </div>
                <div class="tab-pane" id="total1">
                    {df_total_html_unit_1}
                </div>
                <div class="tab-pane" id="total2">
                    {df_total_html_unit_2}
                </div>
            </div>
        </div>
        <!-- Bootstrap JavaScript -->
        <script src="https://code.jquery.com/jquery-3.3.1.slim.min.js"></script>
        <script src="https://stackpath.bootstrapcdn.com/bootstrap/4.3.1/js/bootstrap.min.js"></script>
    </body>
    </html>
    '''

    # HTML 파일 저장
    html_file = 'TPC_GLOBAL.html'
    with open(html_file, 'w') as f:
        f.write(html_template)
        
    return None

def main(file_path):
    xls = pd.ExcelFile(file_path, engine='openpyxl')
    sheets = {sheet_name: xls.parse(sheet_name, header=3) for sheet_name in xls.sheet_names}
    make_dashboard(sheets)

if __name__ == "__main__":
    main('2023_data.xlsx')
    





In [ ]:
file_path = '2023_data.xlsx'
xls = pd.ExcelFile(file_path, engine='openpyxl')
sheets = {sheet_name: xls.parse(sheet_name, header=3) for sheet_name in xls.sheet_names}

# _1 = hr_monthly(sheets)
# _2 = hr_each_month(sheets)
# _3 = pr_monthly(sheets)
# _4 = pr_each_month(sheets)